# Predictions corrections

### table of content


### Dependencies and general utils

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import open3d as o3d
import laspy
import pdal
import json
import pickle
from tqdm import tqdm
from scipy.spatial import cKDTree
from time import time

### Load samples

In [110]:
src_preds = r"..\data\flattening_corrections\predictions"
src_gt = r"..\data\flattening_corrections\gt"
src_floors = r"..\data\flattening_corrections\floors"
src_masks = r"..\data\flattening_corrections\masks"
src_originals = r"..\data\flattening_corrections\originals"
src_flatten = r"..\data\flattening_corrections\flatten"
src_results = r"..\data\flattening_corrections"

In [113]:
list_preds = {}
list_masks = {}
list_floors = {}
list_flatten = {}
list_floors = {}
list_gt = {}
list_originals = {}

samples_num = [128, 129, 160, 210, 311, 633]
tilling_num = [1, 5, 10, 20]
# for num in samples_num:
#     list_preds[num] = []
#     list_masks[num] = []
#     list_floors[num] = []
#     list_originals[num] = []

for r, _, f in os.walk(src_preds):
    for num in samples_num:
        list_preds[num] = [os.path.join(r, file) for file in f if len(file.split(str(num))) > 1]
for r, _, f in os.walk(src_masks):
    for num in samples_num:
        list_masks[num] = [os.path.join(r, file) for file in f if len(file.split(str(num))) > 1]
for r, _, f in os.walk(src_floors):
    for num in samples_num:
        list_floors[num] = [os.path.join(r, file) for file in f if len(file.split(str(num))) > 1]
for r, _, f in os.walk(src_flatten):
    for num in samples_num:
        list_flatten[num] = [os.path.join(r, file) for file in f if len(file.split(str(num))) > 1]
for r, _, f in os.walk(src_originals):
    for num in samples_num:
        list_originals[num] = [os.path.join(r, file) for file in f if len(file.split(str(num))) > 1][0]
for r, _, f in os.walk(src_gt):
    for num in samples_num:
        list_gt[num] = [os.path.join(r, file) for file in f if len(file.split(str(num))) > 1][0]
# print(list_gt)
# print(list_preds)

#     for file in f:
#         list_preds[os.join.path(r, file)] = []
#         list_masks[os.join.path(r, file)] = []
#         list_floors[os.join.path(r, file)] = []
#         list_originals[os.join.path(r, file)] = []

# for r, _, f in os.walk(src_preds):
#     for file in f:

#         # list_originals.append(os.path.join(r,file))
# for r, _, f in os.walk(src_gt):
#     for file in f:
#         list_preds[os.path.join(r, file)] = []
#         list_masks_gt[os.path.join(r, file)] = []
#         list_floors_gt[os.path.join(r, file)] = []
#         # list_originals[os.path.join(r, file)] = []
#         for r_pred, _, f_pred in os.walk(src_preds):
#             for file_pred in f_pred:
#                 if file_pred.split('_flatten')[0].split('_out')[0] == file.split("_out_gt.laz")[0]:
#                     list_preds[os.path.join(r, file)].append(os.path.join(r_pred, file_pred))
#         for r_mask, _, f_mask in os.walk(src_masks):
#             for file_mask in f_mask:
#                 if file_mask.split('_mask')[0].split('_out')[0] == file.split("_out_gt.laz")[0]:
#                     list_masks_gt[os.path.join(r, file)].append(os.path.join(r_mask, file_mask))
#         for r_floor, d_floor, f_floor in os.walk(src_floors):
#             for file_floor in f_floor:
#                 if file_floor.split('_floor')[0].split('_out')[0] == file.split("_out_gt.laz")[0]:
#                     list_floors_gt[os.path.join(r, file)].append(os.path.join(r_floor, file_floor))
#         # for r_original, _, f_original in os.walk(src_originals):
#         #     for file_original in f_original:
#         #         if file_original.split('.laz')[0].split('_flatten')[0] == file.split("_out_gt.laz")[0]:
#         #             list_originals[os.path.join(r, file)].append(os.path.join(r_original, file_original))

# for r, d, f in os.walk(src_gt):
#     for file in f:
#         list_floors.append(os.path.join(r, file))
# print(list_floors_gt)
# print(list_originals)
# for (key, values), (key_2, mask), (key_3, floor) in zip(list_preds_gt.items(), list_masks_gt.items(), list_floors_gt.items()):
# # for key, values in list_preds_gt.items():
#     print(key,":")
#     for pred in values:
#         print("\t", pred)
#     print('\t---')
#     for m in mask:
#         print("\t", m)
#     print('\t---')
#     for f in floor:
#         print("\t", f)
#     print('\t---')


### Metrics

In [92]:
def compute_panoptic_quality(gt_instances, pred_instances):
    """
    Computes Panoptic Quality (PQ), Segmentation Quality (SQ), and Recognition Quality (RQ).
    
    :param gt_instances: List of sets, each containing point indices for a ground truth instance.
    :param pred_instances: List of sets, each containing point indices for a predicted instance.
    :return: PQ, SQ, RQ
    """

    # gt_instances, pred_instances = get_segmentation(gt_instances, pred_instances)
    tp, fp, fn = 0, 0, 0
    iou_sum = 0

    # Match predicted instances to ground truth instances
    matched_gt = set()
    matched_pred = set()
    
    for i, gt in enumerate(gt_instances):
        best_iou = 0
        best_pred = None

        for j, pred in enumerate(pred_instances):
            iou = len(gt & pred) / len(gt | pred)  # IoU computation
            
            if iou > best_iou:
                best_iou = iou
                best_pred = j
        
        # Threshold for a valid match
        if best_iou > 0.5:
            matched_gt.add(i)
            matched_pred.add(best_pred)
            tp += 1
            iou_sum += best_iou
        else:
            fn += 1  # Unmatched ground truth instance
    
    fp = len(pred_instances) - len(matched_pred)  # Unmatched predictions

    RQ = tp / (tp + 0.5 * (fp + fn)) if (tp + 0.5 * (fp + fn)) > 0 else 0
    SQ = iou_sum / tp if tp > 0 else 0
    PQ = SQ * RQ

    return PQ, SQ, RQ, tp, fp, fn


def compute_mean_iou(y_true, y_pred, num_classes=2):
    """
    Computes mean Intersection over Union (mIoU).
    
    :param y_true: Ground truth labels (N,)
    :param y_pred: Predicted labels (N,)
    :param num_classes: Total number of classes
    :return: Mean IoU score
    """
    iou_list = []
    
    for c in range(num_classes):
        tp = np.sum((y_true == c) & (y_pred == c))
        fp = np.sum((y_true != c) & (y_pred == c))
        fn = np.sum((y_true == c) & (y_pred != c))
        
        iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
        iou_list.append(iou)

    return np.mean(iou_list)


def get_segmentation(instance_list, semantic_list):
    instances_format = []
    semantic_format = []
    # Computing instances
    for instance in set(instance_list):
        if instance == 0: continue
        list_points = [pos for pos, val in enumerate(instance_list) if val == instance]
        instances_format.append(set(list_points))

    # Computing semantic
    for semantic in set(semantic_list):
        list_points = [pos for pos, val in enumerate(semantic_list) if val == semantic]
        semantic_format.append(set(list_points))

    return instances_format, semantic_format


### Performances computation

#### Utils

In [93]:


def remove_duplicates(laz_file):
    # Find pairs of points
    coords = np.round(np.vstack((laz_file.x, laz_file.y, laz_file.z)),2).T
    tree_B = cKDTree(coords)
    pairs = tree_B.query_pairs(1e-2)

    # Create the mask with dupplicates
    mask = [True for i in range(len(coords))]
    for pair in pairs:
        mask[pair[1]] = False

    # Remove the dupplicates from the file
    laz_file.points = laz_file.points[mask]

    # # Copy all other attributes (optional)
    # for dim in laz_file.point_format.dimensions:
    #     if dim.name not in ["X", "Y", "Z"]:  # Skip coordinates
    #         setattr(laz_file, dim.name, getattr(laz_file, dim.name)[mask])

def match_pointclouds(laz1, laz2):
    # Retrieve points
    coords_1 = np.round(np.vstack((laz1.x, laz1.y, laz1.z)),2).T
    coords_2 = np.round(np.vstack((laz2.x, laz2.y, laz2.z)),2).T

    # Verify that lengths match:
    assert coords_1.shape == coords_2.shape

    # Convert rows to tuples for sorting
    coords_1_view = np.ascontiguousarray(coords_1).view([('', coords_1.dtype)] * 3)
    coords_2_view = np.ascontiguousarray(coords_2).view([('', coords_2.dtype)] * 3)

    # Sort indices
    coords_1_sort_idx = np.argsort(coords_1_view.ravel())
    coords_2_sort_idx = np.argsort(coords_2_view.ravel())

    # Find corresponding indices
    coords_1_sorted = coords_1_view.ravel()[coords_1_sort_idx]
    coords_2_sorted = coords_2_view.ravel()[coords_2_sort_idx]

    # Get matching indices
    matching_coords_2_idx = np.searchsorted(coords_2_sorted, coords_1_sorted)

    # Restore original indices
    matching_coords_1_idx = coords_1_sort_idx
    matching_coords_2_idx = coords_2_sort_idx[matching_coords_2_idx]
    
    # Assign alligned points to laz files
    # laz1.xyz = coords_1[matching_coords_1_idx]
    # laz2.xyz = coords_2[matching_coords_2_idx]
    # laz1.points = laz1.points[matching_coords_1_idx]
    # laz2.points = laz2.points[matching_coords_2_idx]
    for dim in laz2.point_format.dimensions:  # Skip coordinates
        setattr(laz2, dim.name, getattr(laz2, dim.name)[matching_coords_2_idx])


In [7]:
laz_original = laspy.read(r"..\data\flattening_corrections\originals_old\color_grp_full_tile_128_flatten_1m.laz")
laz_pred = laspy.read(r"..\data\flattening_corrections\predictions\color_grp_full_tile_128_flatten_1m_out.laz")
start = time()
remove_duplicates(laz_original)
print(f"Time to remove duplicates: {int(round(time() - start, 3) * 1000)} ms")
remove_duplicates(laz_pred)
match_pointclouds(laz_original, laz_pred)

Time to remove duplicates: 291 ms


#### Test

In [12]:
gt_src = list(list_preds_gt.keys())[0]
laz_gt = laspy.read(gt_src)
# points_gt = np.vstack((gt.x, gt.y, gt.z)).transpose()

pred_src = list_preds_gt[gt_src][-1]
laz_pred = laspy.read(pred_src)
# points_pred = np.vstack((pred.x, pred.y, pred.z)).transpose()
# print(points_pred.shape)
# print(points_gt.shape)
print(list(laz_gt.point_format.dimension_names))
print(list(laz_pred.point_format.dimension_names))


['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time', 'red', 'green', 'blue', 'PredSemantic', 'PredInstance', 'gt_semantic_segmentation', 'gt_instance_segmentation']
['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time', 'red', 'green', 'blue', 'PredSemantic', 'gt_semantic_segmentation', 'PredInstance']


In [10]:
gt_instances = laz_gt.gt_instance_segmentation
gt_semantic = laz_gt.gt_semantic_segmentation
pred_instances = laz_pred.PredInstance
pred_semantic = laz_pred.PredSemantic

gt_instances_format, gt_semantic_format = get_segmentation(gt_instances, gt_semantic)
pred_instances_format, pred_semantic_format = get_segmentation(pred_instances, pred_semantic)

In [11]:
PQ, SQ, RQ = compute_panoptic_quality(gt_instances_format, pred_instances_format)
mean_iou = compute_mean_iou(laz_gt.gt_semantic_segmentation, laz_pred.PredSemantic)
print(f"PQ: {PQ:.4f}, SQ: {SQ:.4f}, RQ: {RQ:.4f}, mIoU: {mean_iou:.4f}")

ValueError: operands could not be broadcast together with shapes (394889,) (388709,) 

#### Computing on all samples

In [117]:
def match_pointclouds(laz1, laz2):
    """Sort laz2 to match the order of laz1 without changing laz1's order.

    Args:
        laz1: laspy.LasData object (reference order)
        laz2: laspy.LasData object (to be sorted)
    
    Returns:
        laz2 sorted to match laz1
    """
    # Retrieve and round coordinates for robust matching
    coords_1 = np.round(np.vstack((laz1.x, laz1.y, laz1.z)), 2).T
    coords_2 = np.round(np.vstack((laz2.x, laz2.y, laz2.z)), 2).T

    # Verify laz2 is a subset of laz1
    assert len(coords_2) == len(coords_1), "laz2 should be a subset of laz1"

    # Create a dictionary mapping from coordinates to indices
    coord_to_idx = {tuple(coord): idx for idx, coord in enumerate(coords_1)}

    # Find indices in laz1 that correspond to laz2
    matching_indices = []
    failed = 0
    for coord in coords_2:
        try:
            matching_indices.append(coord_to_idx[tuple(coord)])
        except Exception as e:
            failed += 1
    # print(f"Number of non-matching points: {failed}")

    matching_indices = np.array([coord_to_idx[tuple(coord)] for coord in coords_2])

    # Sort laz2 to match laz1
    sorted_indices = np.argsort(matching_indices)

    # Apply sorting to all attributes of laz2
    # for dim in laz2.point_format.dimensions:
    #     setattr(laz2, dim.name, getattr(laz2, dim.name)[sorted_indices])
    laz2.points = laz2.points[sorted_indices]

    return laz2  # Now sorted to match laz1

In [119]:
import copy
import pickle
metrics = ['PQ', 'SQ', 'RQ', 'mIoU', 'Recall', 'Precision']
metrics_res = {metric: np.zeros((len(samples_num),len(tilling_num))) for metric in metrics}

for i, samp_num in tqdm(enumerate(samples_num), total=len(samples_num)):
    # print(f"PROCESSING SAMPLE {samp_num}")
    original_src = list_originals[samp_num]
    gt_src = list_gt[samp_num]
    laz_original = laspy.read(original_src)

    for j, tilling in enumerate(tilling_num):
        pred_src = [x for x in list_preds[samp_num] if len(x.split(f'{tilling}m')) > 1][0]
        floor_src = [x for x in list_floors[samp_num] if len(x.split(f'{tilling}m')) > 1][0]
        flatten_src = [x for x in list_flatten[samp_num] if len(x.split(f'{tilling}m')) > 1][0]
        mask_src = [x for x in list_masks[samp_num] if len(x.split(f'{tilling}m')) > 1][0]

        laz_pred = laspy.read(pred_src)
        laz_gt = laspy.read(gt_src)
        laz_floor = laspy.read(floor_src)
        laz_flatten = laspy.read(flatten_src)

        laz_pred = match_pointclouds(laz_flatten, laz_pred)

        pred_coords = np.vstack((laz_pred.x, laz_pred.y, laz_pred.z)).T
        gt_coords = np.vstack((laz_gt.x, laz_gt.y, laz_gt.z)).T
        floor_coords = np.vstack((laz_floor.x, laz_floor.y, laz_floor.z)).T

        remove_duplicates(laz_gt)
        laz_gt = match_pointclouds(laz_original, laz_gt)

        # Crop groud truth
        if len(pred_src.split('flatten')) > 1:
            # add floor to preds
            pred_coords[:,2] = pred_coords[:,2] + floor_coords[:,2]
            setattr(laz_pred, 'x', pred_coords[:,0])
            setattr(laz_pred, 'y', pred_coords[:,1])
            setattr(laz_pred, 'z', pred_coords[:,2])

            # load mask
            with open(mask_src, 'rb') as infile:
                mask = pickle.load(infile)
            laz_gt.points = laz_gt.points[mask]
            laz_gt = match_pointclouds(laz_pred, laz_gt)

        # Compute metrics
        gt_instances = laz_gt.gt_instance_segmentation
        gt_semantic = laz_gt.gt_semantic_segmentation
        pred_instances = laz_pred.PredInstance
        pred_semantic = laz_pred.PredSemantic

        gt_instances_format, gt_semantic_format = get_segmentation(gt_instances, gt_semantic)
        pred_instances_format, pred_semantic_format = get_segmentation(pred_instances, pred_semantic)
        
        PQ, SQ, RQ, tp, fp, fn = compute_panoptic_quality(gt_instances_format, pred_instances_format)
        mean_iou = compute_mean_iou(gt_semantic, pred_semantic)
        metrics_res['PQ'][i,j] = PQ
        metrics_res['SQ'][i,j] = SQ
        metrics_res['RQ'][i,j] = RQ
        metrics_res['mIoU'][i,j] = mean_iou
        metrics_res['Recall'][i,j] = round(tp/(tp + fn), 2) if tp + fn > 0 else 0
        metrics_res['Precision'][i,j] = round(tp/(tp + fp),2) if tp + fn > 0 else 0





# # print(metrics_res)
# arr_metric_res = np.zeros((6*5, 4))
# # for i in tqdm(range(arr_metric_res.shape[0]), total=arr_metric_res.shape[0]):
# for i in range(arr_metric_res.shape[0]):
#     # Load pointclouds
#     gt_src = list(list_preds_gt.keys())[i//6]
#     # pred_src = list_preds_gt[gt_src][i%5]
#     # floor_src = list_floors_gt[gt_src][i%5]
#     original_full_src = list_originals[i//6]
#     pred_src = "../data/flattening_corrections/test/color_grp_full_tile_128_flatten_10m_out.laz"
#     floor_src = "../data/flattening_corrections/test/color_grp_full_tile_128_floor_10m.laz"
#     original_src = "../data/flattening_corrections/test/color_grp_full_tile_128_flatten_10m.laz"
#     laz_pred = laspy.read(pred_src)
#     laz_gt = laspy.read(gt_src)
#     laz_floor = laspy.read(floor_src)
#     laz_original = laspy.read(original_src)
#     laz_original_full = laspy.read(original_full_src)

#     laz_pred = match_pointclouds(laz_original, laz_pred)
#     print("Original: ", original_src)
#     print("GT: ", gt_src)
#     print("Pred: ", pred_src)
#     print("Floor: ", floor_src)
#     pred_coords = np.vstack((laz_pred.x, laz_pred.y, laz_pred.z)).T
#     gt_coords = np.vstack((laz_gt.x, laz_gt.y, laz_gt.z)).T
#     floor_coords = np.vstack((laz_floor.x, laz_floor.y, laz_floor.z)).T

    
    

#     remove_duplicates(laz_gt)
#     laz_gt = match_pointclouds(laz_original_full, laz_gt)

#     # laz_gt = match_pointclouds(laz_original, laz_gt)
#     # for i in tqdm(range(len(laz_gt))):
#     #     if not (laz_gt.xyz[i,:] == laz_original.xyz[i,:]).all(axis=0):
#     #         print("YARGH")
#     print(f"Is gt equal original : {np.array_equal(np.vstack(np.round((laz_gt.x, laz_gt.y, laz_gt.z),2)), np.vstack(np.round((laz_original.x, laz_original.y, laz_original.z),2)))}")

#     if round(laz_gt.z[748],2) != round(laz_original.z[748],2):
#         print("YARGH")
#     # gt_coords = np.vstack((laz_gt.x, laz_gt.y, laz_gt.z)).T

#     # Crop groud truth
#     if len(pred_src.split('flatten')) > 1:
#         # add floor to preds
#         pred_coords[:,2] = pred_coords[:,2] + floor_coords[:,2]
#         setattr(laz_pred, 'x', pred_coords[:,0])
#         setattr(laz_pred, 'y', pred_coords[:,1])
#         setattr(laz_pred, 'z', pred_coords[:,2])
#         laz_pred.write('../data/test.laz')

#         # load mask
#         mask_src = list_masks_gt[gt_src][i%5]
#         with open(mask_src, 'rb') as infile:
#             mask = pickle.load(infile)
#         laz_gt.points = laz_gt.points[mask]
#         # laz_original.points = laz_original.points[mask]
#         # laz_original = match_pointclouds(laz_pred, laz_original)
#         laz_gt = match_pointclouds(laz_pred, laz_gt)
#         print(laz_original.xyz[7648,:])
#         print(len(laz_pred))
#         print(len(laz_gt))
#         # laz_gt = match_pointclouds(laz_pred, laz_gt)
#         print(len(laz_gt))
#         # laz_gt.xyz -= laz_floor.xyz
#     # print(pred_coords[7648,:])
#     print(laz_gt.xyz[7648,:])
#     print(laz_pred.xyz[7648,:])
#     # print(laz_pred.z[7648])
#     # print(laz_floor.xyz[7648,:])
#     print(f"Is gt equal pred : {np.array_equal(np.vstack(np.round((laz_gt.x, laz_gt.y, laz_gt.z),2)), np.vstack(np.round((laz_pred.x, laz_pred.y, laz_pred.z),2)))}")
#     # print(f"Is gt equal pred : {np.array_equal(np.round(laz_gt.xyz,2), np.round(laz_pred.xyz,2))}")

#     # Compute metrics
#     gt_instances = laz_gt.gt_instance_segmentation
#     gt_semantic = laz_gt.gt_semantic_segmentation
#     pred_instances = laz_pred.PredInstance
#     pred_semantic = laz_pred.PredSemantic

#     gt_instances_format, gt_semantic_format = get_segmentation(gt_instances, gt_semantic)
#     pred_instances_format, pred_semantic_format = get_segmentation(pred_instances, pred_semantic)

#     # PQ, SQ, RQ = compute_panoptic_quality(gt_instances_format, pred_instances_format)
#     # mean_iou = compute_mean_iou(laz_gt.gt_semantic_segmentation, laz_pred.PredSemantic)
    
#     PQ, SQ, RQ, tp, fp, fn = compute_panoptic_quality(gt_instances_format, pred_instances_format)
#     mean_iou = compute_mean_iou(gt_semantic, pred_semantic)

#     print(set(gt_instances))
#     print(set(gt_semantic))
#     print(set(pred_instances))
#     print(set(pred_semantic))


#     print(f"PQ: {PQ:.4f}, SQ: {SQ:.4f}, RQ: {RQ:.4f}, tp: {tp}, fp: {fp}, fn: {fn}, Recall: {round(tp/(tp + fn), 2)}, Precision: {round(tp/(tp + fp),2)}, mIoU: {mean_iou:.4f}")
#     break
    

100%|██████████| 6/6 [08:16<00:00, 82.71s/it] 


In [122]:
for metric in metrics_res.keys():
    print(metric)
    print(metrics_res[metric])

PQ
[[0.27893734 0.34397479 0.30623971 0.3653979 ]
 [0.01545159 0.00774913 0.0076665  0.        ]
 [0.         0.         0.         0.        ]
 [0.40451977 0.55216285 0.33103448 0.56080114]
 [0.48505885 0.54665073 0.54924323 0.50812442]
 [0.30352201 0.31660588 0.29101086 0.28828947]]
SQ
[[0.85424559 0.87904667 0.90034476 0.89944099]
 [0.64896697 0.68967276 0.69765143 0.        ]
 [0.         0.         0.         0.        ]
 [0.60677966 0.82824427 0.82758621 0.84120172]
 [0.87683715 0.87416992 0.88378229 0.86443117]
 [0.72727028 0.71456188 0.70768551 0.69629237]]
RQ
[[0.32653061 0.39130435 0.34013605 0.40625   ]
 [0.02380952 0.01123596 0.01098901 0.        ]
 [0.         0.         0.         0.        ]
 [0.66666667 0.66666667 0.4        0.66666667]
 [0.55319149 0.62533693 0.62146893 0.58781362]
 [0.41734417 0.44307692 0.41121495 0.41403509]]
mIoU
[[0.63191958 0.63392697 0.63583162 0.65339008]
 [0.35938154 0.3375126  0.32546136 0.3116106 ]
 [0.45848743 0.45757023 0.44668689 0.476849

In [ ]:
arr_metric_res = np.zeros((6*5, 4))
for i in tqdm(range(arr_metric_res.shape[0]), total=arr_metric_res.shape[0]):
    i=1
    gt_src = list(list_preds_gt.keys())[i//5]
    floor_src = list_floors[i//5]
    if len(list_preds_gt[gt_src]) < i % 5:
        continue
    pred_src = list_preds_gt[gt_src][i%5]
    floor_src = list_floors_gt[gt_src][i%5]
    original_src = list_originals[gt_src][i+1%5]
    # print(pred_src)
    # print(floor_src)
    # print(original_src)
    # print(gt_src)
    # load files and addapt gt to pred size
    laz_gt = laspy.read(gt_src)
    laz_pred = laspy.read(pred_src)
    laz_floor = laspy.read(floor_src)
    laz_original = laspy.read(original_src)

    print(len(laz_pred))
    print(len(laz_original))
    print(len(laz_floor))
    remove_duplicates(laz_pred)
    remove_duplicates(laz_original)
    remove_duplicates(laz_floor)
    
    print(len(laz_pred))
    print(len(laz_original))
    print(len(laz_floor))
    break
    with open(list_masks_gt[gt_src][(i-1)%5], 'rb') as file:
        mask = pickle.load(file)

    # load files and addapt gt to pred size
    laz_gt = laspy.read(gt_src)
    laz_pred = laspy.read(pred_src)
    laz_floor = laspy.read(floor_src)
    print(floor_src)

    # Extract (x, y, z) coordinates as Nx3 arrays
    coords_gt = np.round(np.vstack((laz_gt.x, laz_gt.y)),2).T
    # coords_gt = coords_gt[mask]
    coords_pred = np.round(np.vstack((laz_pred.x, laz_pred.y, laz_pred.z)),2).T
    print(coords_gt.shape)
    print(coords_pred.shape)

    original = laspy.read(r"..\data\flattening_corrections\originals\color_grp_full_tile_128_flatten_1m.laz")
    coords_original = np.round(np.vstack((original.x, original.y, original.z)),2).T
    counter = 0
    for row in tqdm(range(coords_original.shape[0]), total=coords_original.shape[0]):
        matches = np.isin(coords_pred, coords_original[row,:]).all(axis=1)
        counter += np.sum(matches)
    print(counter)
    break
    print("original: ", len(original))
    print("gt: ", len(laz_gt))
    print("pred: ", len(laz_pred))
    print("mask: ", len(mask))
    print("floor: ", len(laz_floor))


    tree_B = cKDTree(coords_pred)
    print("computing distances...")
    distances, indices = tree_B.query(original, distance_upper_bound=1e-2, workers=-1)
    print('done')
    # pairs = tree_B.query_pairs(5e-2)
    mask = distances != np.inf
    coords_pred = coords_pred[mask]
    # laz_gt = laz_gt.points[mask]
    print("masked pred", len(coords_pred))
    # print(len(pairs))

    break
    # print(min(coords_gt[:,2]))
    # print(min(coords_pred[:,2]))
    # coords_pred[:,2] += np.min(coords_gt[:2])
    # # Find the points in A that exist in B
    # mask = np.isin(coords_gt.view([('', coords_gt.dtype)] * 3), 
    #             coords_pred.view([('', coords_pred.dtype)] * 3)).all(axis=1)

    # # Filter A to keep only matching points
    # laz_gt = laz_gt.points[mask]

    tree_B = cKDTree(coords_pred)
    distances, indices = tree_B.query(coords_gt, distance_upper_bound=1e-2)
    mask = distances != np.inf
    laz_gt = laz_gt.points[mask]

    # Compute the metrics
    gt_instances_format, gt_semantic_format = get_segmentation(laz_gt.gt_instance_segmentation, laz_pred.PredInstance)
    PQ, SQ, RQ = compute_panoptic_quality(gt_instances_format, pred_instances_format)
    mean_iou = compute_mean_iou(laz_gt.gt_semantic_segmentation, laz_pred.PredSemantic)

    # save metrics
    arr_metric_res[i,:] = np.array([PQ, SQ, RQ, mean_iou])
    break
    

  0%|          | 0/30 [00:00<?, ?it/s]

..\data\flattening_corrections\predictions\color_grp_full_tile_128_flatten_1m_out.laz
..\data\flattening_corrections\floors\color_grp_full_tile_128_floor_1m.laz
..\data\flattening_corrections\originals\color_grp_full_tile_128_flatten_1m.laz
..\data\flattening_corrections\gt\color_grp_full_tile_128_out_gt.laz
392648
388464
388464


  0%|          | 0/30 [00:00<?, ?it/s]

386579
386579
366718


In [83]:
laz_original = laspy.read(r"..\data\flattening_corrections\originals\color_grp_full_tile_128_flatten_1m.laz")
laz_pred = laspy.read(r"..\data\flattening_corrections\predictions\color_grp_full_tile_128_flatten_1m_out.laz")
coords_pred = np.round(np.vstack((laz_pred.x, laz_pred.y, laz_pred.z)),2).T
coords_original = np.round(np.vstack((original.x, original.y, original.z)),2).T
print("original shape: ", len(coords_original))
print("original shape: ", len(coords_pred))

counter = []
nomatches = []
multimatches = []
for row in tqdm(range(coords_original.shape[0]), total=coords_original.shape[0]):
    matches = np.isin(coords_pred, coords_original[row,:]).all(axis=1)
    count = np.sum(matches)
    counter.append(count)
    if count == 0:
        nomatches.append(row)
    elif count > 1:
        multimatches.append(row)
    # if row == 1000:
    #     break

for val in set(counter):
    print(f"Number of val repeated {val}x: {len([x for x in counter if x == val])}")

original shape:  388464
original shape:  392648


100%|██████████| 388464/388464 [1:20:18<00:00, 80.62it/s]

Number of val repeated 8x: 204
Number of val repeated 1x: 384697
Number of val repeated 4x: 3554
Number of val repeated 9x: 9


In [84]:
duplicated = {
    "counter": counter,
    "nomatches": nomatches,
    "multi": multimatches
}
with open("../data/flattening_corrections/duplicated_vals_128_1.pcl", 'wb') as file:
    pickle.dump(duplicated, file)

In [138]:
count_same = 0
non_matching_pred = []
non_matching_original = []
for row in tqdm(range(coords_pred.shape[0])):
    # print(row)
    # if np.round(coords_original[row,:], 2).all(axis=0) == np.round(coords_pred[row,:], 2).all(axis=0):
    if np.any(np.all(coords_original == coords_pred[row,:], axis=1)):
        count_same += 1
    else:
        non_matching_pred.append(np.round(coords_pred[row,:], 2))
        non_matching_original.append(np.round(coords_original[row,:], 2))
print(count_same)

100%|██████████| 386579/386579 [30:23<00:00, 212.01it/s]

386579


In [17]:
# Sort the two arrays

# # Example arrays (shuffled versions of each other)
# A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
# B = np.array([[7, 8, 9], [1, 2, 3], [4, 5, 6]])

A = coords_original
B = coords_pred
# Convert rows to tuples for sorting
A_view = np.ascontiguousarray(A).view([('', A.dtype)] * 3)
B_view = np.ascontiguousarray(B).view([('', B.dtype)] * 3)

# Sort indices
A_sort_idx = np.argsort(A_view.ravel())
B_sort_idx = np.argsort(B_view.ravel())

# Find corresponding indices
A_sorted = A_view.ravel()[A_sort_idx]
B_sorted = B_view.ravel()[B_sort_idx]

# Get matching indices
matching_B_idx = np.searchsorted(B_sorted, A_sorted)

# Restore original indices
matching_A_idx = A_sort_idx
matching_B_idx = B_sort_idx[matching_B_idx]

# Reorganise B
coords_pred = coords_pred[matching_B_idx]
coords_original = coords_original[matching_A_idx]

# Print results
print("A indices:", matching_A_idx)
print("B indices:", matching_B_idx)


A indices: [     0      1      2 ... 386576 386577 386578]
B indices: [     0      1      2 ... 386576 386577 386578]


In [ ]:
# Test if both pointclouds matches
for i in tqdm(range(coords_original.shape[0])):
    if not (coords_original[i,:] == coords_pred[i,:]).all(axis=0):
        print("YARGH")

  0%|          | 0/386579 [00:00<?, ?it/s]

100%|██████████| 386579/386579 [00:00<00:00, 433692.08it/s]


In [123]:
count_diff = 0
really_non_matching = []
for _, row in tqdm(enumerate(non_matching_pred), total=len(non_matching_pred)):
    found = False
    for other_row in non_matching_original:
        if np.array_equal(row, other_row):
            found = True
            break
    if found == False:
        count_diff += 1
        really_non_matching.append(row)

print(count_diff)

  5%|▍         | 1478/32394 [01:12<25:26, 20.25it/s]


KeyboardInterrupt: 

### Other stuff

In [ ]:
# Take predictions out of the results folders
from glob import glob
src_root = r"..\data\flattening_predictions_corrections\predictions"
for filename in glob(os.path.join(src_root, '**/*.laz'), recursive=True):
    os.replace(filename, os.path.join(src_root, os.path.basename(filename)))
# for r, d, f in os.walk(src_root):
#     print(r)
#     print(d)
#     print(f)
#     break